###숙소 추천

랜덤 포레스트 모델

모델 학습

In [1]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

# CSV 파일 경로
file_path = '/content/Compilation_travel_data.csv'  # 파일 경로를 수정하세요

# CSV 파일 로드
df = pd.read_csv(file_path)

# 데이터 준비
x = df[['GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_4',
         'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_8', 'TRAVEL_COMPANIONS_NUM']]
y = df['LODGING_TYPE_CD']

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 모델 성능 비교를 위한 변수 초기화
best_accuracy = 0
best_model = None

# 다양한 n_estimators 테스트
for n in [10, 30, 50, 100]:
    model = RandomForestClassifier(n_estimators=n, random_state=42)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"n_estimators={n}: Accuracy={accuracy}, F1 Score={f1}")

    # 가장 높은 정확도의 모델 저장
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model

# 최상의 모델 저장
if best_model is not None:
    with open('best_random_forest_model.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    print("Best model saved with accuracy:", best_accuracy)

# 결과 출력
result = pd.DataFrame({'pred': y_pred, 'result': y_test})
result['matching'] = (result['pred'] == result['result'])
print(result)


n_estimators=10: Accuracy=0.7546134663341646, F1 Score=0.7506988284622881
n_estimators=30: Accuracy=0.7536159600997506, F1 Score=0.7500107807317692
n_estimators=50: Accuracy=0.7531172069825436, F1 Score=0.7507186466253559
n_estimators=100: Accuracy=0.7531172069825436, F1 Score=0.7500578630311882
Best model saved with accuracy: 0.7546134663341646
      pred  result  matching
3543   3.0     3.0      True
5731   1.0     1.0      True
4801   3.0     1.0     False
416    7.0     7.0      True
5125   1.0     1.0      True
...    ...     ...       ...
5434   3.0    10.0     False
1817   1.0     1.0      True
2625   1.0     4.0     False
5130   3.0     3.0      True
8906   1.0     1.0      True

[2005 rows x 3 columns]


모델로 입력값에 따른 숙소 유형 추천

In [2]:
import pickle
import pandas as pd

# 모델 로드
with open('best_random_forest_model.pkl', 'rb') as f:
    model = pickle.load(f)

# 추천 함수 정의
def recommend_lodging(gender, age_grp, travel_styl_1, travel_styl_4,
                      travel_styl_5, travel_styl_6, travel_styl_8, travel_companions_num):
    # 성별과 연령대를 숫자로 변환
    # 남자: 0, 여자: 1 로 변환
    gender_mapping = {'남자': 0, '여자': 1}
    gender = gender_mapping.get(gender, -1) # If gender is not found, set to -1

    # 연령대를 숫자로 변환 (예: '10' -> 10)
    try:
        age_grp = int(age_grp)
    except ValueError:
        age_grp = -1 # If age_grp cannot be converted to int, set to -1

    # 입력 데이터를 데이터프레임으로 변환
    input_data = pd.DataFrame({
        'GENDER': [gender],
        'AGE_GRP': [age_grp],
        'TRAVEL_STYL_1': [travel_styl_1],
        'TRAVEL_STYL_4': [travel_styl_4],
        'TRAVEL_STYL_5': [travel_styl_5],
        'TRAVEL_STYL_6': [travel_styl_6],
        'TRAVEL_STYL_8': [travel_styl_8],
        'TRAVEL_COMPANIONS_NUM': [travel_companions_num]
    })

    # 모델을 사용하여 예측
    prediction = model.predict(input_data)

    return prediction[0]  # 첫 번째 예측값 반환


# 사용자 입력 받기
gender = input("성별을 입력하세요 (남자/여자): ")
age_grp = input("연령대를 입력하세요 (예: 10, 20, 30 ...): ")

print('0~7까지의 값 입력')
travel_styl_1 = int(input("자연 vs 도시: "))
travel_styl_4 = int(input("편하지만 비싼 숙소 vs 저렴 : "))
travel_styl_5 = int(input("휴식 vs 체험활동 (0-7): "))
travel_styl_6 = int(input("알려지지 않은 방문지 vs 알려진 방문지 (0-7): "))
travel_styl_8 = int(input("사진촬영: "))
travel_companions_num = int(input("동행자 수를 입력하세요(본인포함x): "))

# 숙소 추천 호출
recommended_lodging = recommend_lodging(gender, age_grp,
                                         travel_styl_1, travel_styl_4,
                                         travel_styl_5, travel_styl_6,
                                         travel_styl_8, travel_companions_num)

print(f"추천 숙소 유형: {recommended_lodging}")

성별을 입력하세요 (남자/여자): 남자
연령대를 입력하세요 (예: 10, 20, 30 ...): 20
0~7까지의 값 입력
자연 vs 도시: 5
편하지만 비싼 숙소 vs 저렴 : 6
휴식 vs 체험활동 (0-7): 1
알려지지 않은 방문지 vs 알려진 방문지 (0-7): 3
사진촬영: 2
동행자 수를 입력하세요(본인포함x): 0
추천 숙소 유형: 7.0


숙소 유형에 따른 숙소 추천 5개

In [4]:
import random
def get_random_lodging_names(lodging_type):
    # CSV 파일 로드
    df = pd.read_csv('Compilation_travel_data.csv')

    # 해당 LODGING_TYPE_CD의 숙소 이름 필터링
    matching_lodgings = df[df['LODGING_TYPE_CD'] == lodging_type]['LODGING_NM'].tolist()

    # 랜덤으로 5개 선택
    random_lodgings = random.sample(matching_lodgings, min(5, len(matching_lodgings)))

    return random_lodgings
# 추천된 숙소 유형에 따라 숙소 이름 랜덤 선택
random_lodging_names = get_random_lodging_names(recommended_lodging)

print("추천된 숙소 이름:")
for lodging in random_lodging_names:
    print(lodging)

추천된 숙소 이름:
모들공간
토다게스트하우스
점보네게스트하우스
제주사교장
제주사교장
